In [12]:
from llama_index.llms.together import TogetherLLM
from llama_index.llms.openrouter import OpenRouter
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)
from llama_index.core.llms import ChatMessage

In [37]:
def togetherLLM():
    return TogetherLLM(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
        api_key="4549dec9a1eb1cd1c79aa2e8501cf3c639b8a75a281f788036267f96c48746ab",
        is_chat_model=True,
        default_headers={},
        logprobs=1,
        num_output=2048
    )

In [2]:
embeddings = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")

In [3]:
vector_store = MilvusVectorStore(
    uri="http://localhost:19530",
    collection_name="lms",
    dim=768,
    index_config={
        "metric_type": "IP",  # Inner Product untuk similarity
        "index_type": "IVF_FLAT",  # Lebih efisien untuk dataset besar
        "params": {"nlist": 128},  # Parameter index
    },
)

2025-03-20 09:17:58,786 [DEBUG][_create_connection]: Created new connection using: 21c32f91ab4444adb82c36d9eece5e93 (async_milvus_client.py:600)


In [4]:
def openRouterDeepseek():
    return OpenRouter(
        model="deepseek/deepseek-chat:free",
        api_key="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",
        context_window=4096,
        max_tokens=5056
    )

In [5]:
def metadataFilter(key, value, operator="=="):
    """ "
    # TODO add more operators
    EQ = "=="  # default operator (string, int, float)
    GT = ">"  # greater than (int, float)
    LT = "<"  # less than (int, float)
    NE = "!="  # not equal to (string, int, float)
    GTE = ">="  # greater than or equal to (int, float)
    LTE = "<="  # less than or equal to (int, float)
    IN = "in"  # metadata in value array (string or number)
    NIN = "nin"  # metadata not in value array (string or number)
    TEXT_MATCH = "text_match"  # full text match (allows you to search for a specific substring, token or phrase within the text field)
    CONTAINS = "contains"  # metadata array contains value (string or number)
    """
    getOperator = {
        "==": FilterOperator.EQ,
        ">": FilterOperator.GT,
        "<": FilterOperator.LT,
        "!=": FilterOperator.NE,
        ">=": FilterOperator.GTE,
        "<=": FilterOperator.LTE,
        "in": FilterOperator.IN,
        "nin": FilterOperator.NIN,
        "text_match": FilterOperator.TEXT_MATCH,
        "contains": FilterOperator.CONTAINS,
    }
    return MetadataFilters(
        filters=[
            MetadataFilter(
                key=key, operator=getOperator[operator.lower()], value=value
            ),
        ]
    )

In [15]:
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embeddings
)

In [6]:
# llm = togetherLLM()
llm = openRouterDeepseek()

In [7]:
Settings.llm = openRouterDeepseek()
Settings.embed_model = embeddings

In [16]:
from pydantic import BaseModel, Field, model_validator
from typing import List, Optional

class Answer(BaseModel):  
    title: str = Field(..., description="The title of the answer.")  
    correct: int = Field(..., description="Indicates whether the answer is correct (1) or not (0). Shuffle the correct answer for each question.")  

class Question(BaseModel):  
    title: str = Field(..., description="The title of the question.")  
    type: str = Field(..., description="The type of the question (e.g., multiple, descriptive).")  
    descriptive_correct_answer: Optional[str] = Field(None, description="The descriptive correct answer, if applicable.")  
    grade: int = Field(..., description="The grade or score assigned to the question.")  
    answers: List[Answer] = Field(..., description="A list of possible answers for the question. There must be exactly 4 answers (A, B, C, D).")  
    
    @model_validator(mode="before")
    def validate_fields_based_on_type(cls, values):  
        question_type = values.get("type")
        descriptive_correct_answer = values.get("descriptive_correct_answer")  
        answers = values.get("answers")  

        if question_type == "multiple":  
            if descriptive_correct_answer is not None:  
                raise ValueError("descriptive_correct_answer must be None for multiple-choice questions.")  
            if not answers:  
                raise ValueError("answers must be provided for multiple-choice questions.")  
        else:  
            if not descriptive_correct_answer:  
                raise ValueError("descriptive_correct_answer must be provided for non-multiple-choice questions.")  
            if answers:  
                raise ValueError("answers must be empty for non-multiple-choice questions.")  
        return values

class QuizOutput(BaseModel):  
    lang: str = Field(..., description="The language of the response.")  
    quiz_id: int = Field(..., description="The ID of the quiz.")  
    questions: List[Question] = Field(..., description="A list of questions in the quiz.")

class QueryInput(BaseModel):
    query: str = Field(..., description="The query to be processed.")

class QueryOutput(BaseModel):
    result: List[QueryInput] = Field(..., description="The result of the query.")

In [17]:
from typing import List
from pydantic import BaseModel, Field

class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_seconds: int


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]

In [18]:
sllm = llm.as_structured_llm(output_cls=QueryOutput)

result = await sllm.achat([
    ChatMessage(role="system", content="Anda adalah asisten yang membantu mencarikan materi untuk membuat soal"),
    ChatMessage(role="user", content="Buat query pencarian untuk materi tentang Quantum, Newton yang cocok untuk pembuatan soal ujian. Fokus pada konsep utama dan variasi pertanyaan"),
])

In [19]:
print(result.raw)

result=[QueryInput(query='Konsep dasar mekanika kuantum: prinsip ketidakpastian Heisenberg, fungsi gelombang, dan dualitas partikel-gelombang.'), QueryInput(query='Hukum gerak Newton: hukum inersia, hukum aksi-reaksi, dan hukum percepatan.'), QueryInput(query='Perbandingan antara fisika klasik (Newton) dan fisika kuantum: perbedaan dalam skala, determinisme, dan interpretasi.'), QueryInput(query='Aplikasi hukum Newton dalam kehidupan sehari-hari: contoh gerak benda, gaya gesek, dan gerak melingkar.'), QueryInput(query='Eksperimen kuantum terkenal: celah ganda, efek fotolistrik, dan paradoks EPR.'), QueryInput(query='Soal latihan: hitung energi foton menggunakan persamaan Planck dan jelaskan hubungannya dengan efek fotolistrik.'), QueryInput(query='Soal latihan: analisis gerak benda pada bidang miring menggunakan hukum Newton dan hitung percepatannya.'), QueryInput(query='Soal latihan: jelaskan prinsip superposisi dalam mekanika kuantum dan berikan contohnya.'), QueryInput(query='Soal l

In [8]:
sllm = llm.as_structured_llm(output_cls=QueryOutput)
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="compact",
    filter=metadataFilter("material_id", [152, 150], "in")
)

NameError: name 'QueryOutput' is not defined

In [64]:
index.as_query_engine(llm=llm).query("halo")

TypeError: 'NoneType' object is not subscriptable

In [13]:
result = await query_engine.aquery("Konsep dasar riwayat budi dan penerapannya dalam analisis perilaku")
result

NameError: name 'query_engine' is not defined

In [14]:
await llm.achat([
    ChatMessage(role="system", content="Anda adalah asisten yang membantu mencarikan materi untuk membuat soal"),
    ChatMessage(role="user", content="Buat query pencarian untuk materi tentang Quantum, Newton yang cocok untuk pembuatan soal ujian. Fokus pada konsep utama dan variasi pertanyaan")
    ])

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Berikut adalah beberapa query pencarian yang dapat Anda gunakan untuk mencari materi tentang **Quantum** dan **Newton** yang cocok untuk pembuatan soal ujian. Query ini difokuskan pada konsep utama dan variasi pertanyaan:\n\n### 1. **Quantum Mechanics**\n   - "Basic principles of quantum mechanics for exam questions"\n   - "Quantum superposition and entanglement explained for students"\n   - "Heisenberg uncertainty principle application problems"\n   - "Wave-particle duality examples and questions"\n   - "Quantum tunneling concept and practice problems"\n   - "Schrödinger equation and its solutions for exams"\n   - "Quantum states and operators explained with examples"\n   - "Quantum harmonic oscillator problems for students"\n   - "Applications of quantum mechanics in technology"\n   - "Quantum measurement problem and its implications"\n\n### 2. 

In [9]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

completion = client.chat.completions.create(
  # extra_headers={
  #   "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
  #   "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  # },
  model="deepseek/deepseek-chat:free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)

print(completion.choices[0].message.content)


The question "What is the meaning of life?" is one of the most profound and enduring philosophical inquiries. Different cultures, religions, philosophies, and individuals have offered various interpretations over the centuries. Here are some perspectives:

1. **Religious Views**:
   - **Christianity**: The meaning of life is to love God and follow His commandments, with the ultimate goal of eternal life in heaven.
   - **Islam**: Life's purpose is to worship Allah and live according to His will, aiming for paradise in the afterlife.
   - **Buddhism**: The goal is to achieve enlightenment (Nirvana) by overcoming suffering and the cycle of rebirth.
   - **Hinduism**: Life's purpose is to fulfill one's dharma (duty) and achieve moksha (liberation from the cycle of rebirth).

2. **Philosophical Views**:
   - **Existentialism**: Life has no inherent meaning, and it is up to each individual to create their own purpose through choices and actions.
   - **Stoicism**: The meaning of life is to 